In [125]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
%matplotlib inline
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics

import requests

In [199]:
data = pd.read_csv('http://3e9387c7.ngrok.io/get_data')

In [200]:
data

,x,y,z
0,0.000000,0.000000,0.000000
1,-0.628174,0.007812,0.618164
2,-0.294922,-0.438721,1.981689
3,2.098877,0.306885,-2.428223
4,-1.146240,-0.264648,1.073242
...,...,...,...
2486,1.328857,-0.136475,-0.475586
2487,-0.407471,-0.293701,1.142578
2488,1.362793,-0.244873,-0.478760
2489,-0.398193,-0.047607,0.041016


In [201]:
# data.to_csv('gesture_decline.csv')

In [205]:
gesture1 = pd.read_csv('gesture_accept.csv')
gesture2 = pd.read_csv('gesture_decline.csv')

#### training data prepocessing 

In [207]:
gesture2

,Unnamed: 0,x,y,z
0,0,0.000000,0.000000,0.000000
1,1,-0.628174,0.007812,0.618164
2,2,-0.294922,-0.438721,1.981689
3,3,2.098877,0.306885,-2.428223
4,4,-1.146240,-0.264648,1.073242
...,...,...,...,...
2486,2486,1.328857,-0.136475,-0.475586
2487,2487,-0.407471,-0.293701,1.142578
2488,2488,1.362793,-0.244873,-0.478760
2489,2489,-0.398193,-0.047607,0.041016


In [208]:
# 1 = accept, 2 = decline

gesture1 = gesture1.drop(columns=['Unnamed: 0'])
gesture2 = gesture2.drop(columns=['Unnamed: 0'])

gesture1['label'] = 1
gesture2['label'] = 2

gesture1 = gesture1.loc[:3000]
gesture2 = gesture2.loc[:2490]

dataset = gesture1.append(gesture2)

### Preprocessing 1

In [211]:
drop_zeros = dataset.loc[(dataset==0).any(1)].index
dataset = dataset.drop(drop_zeros, axis=0)
dataset = dataset.reset_index(drop=True)

labels = dataset['label'].to_numpy()
labels = labels[:-10:10]

dataset = dataset[['x','y','z']]

In [214]:
labels.size

546

In [215]:
dataset

,x,y,z
0,-0.064209,0.028076,0.154785
1,-0.645020,-0.220459,0.068604
2,2.791260,-0.025879,0.772705
3,-0.729736,-0.090332,0.064453
4,-0.746582,-0.261719,0.439941
...,...,...,...
5463,1.328857,-0.136475,-0.475586
5464,-0.407471,-0.293701,1.142578
5465,1.362793,-0.244873,-0.478760
5466,-0.398193,-0.047607,0.041016


In [216]:
dataset = dataset.loc[:5460]

In [217]:
dataset

,x,y,z
0,-0.064209,0.028076,0.154785
1,-0.645020,-0.220459,0.068604
2,2.791260,-0.025879,0.772705
3,-0.729736,-0.090332,0.064453
4,-0.746582,-0.261719,0.439941
...,...,...,...
5456,2.004395,-0.074951,-0.950439
5457,0.312500,-0.072021,0.212891
5458,0.654541,-0.077148,0.554199
5459,1.700684,0.116211,-1.566406


### Preprocessing 2

In [240]:
frames = []
def frame():
    n=0
    while n != len(dataset)-1:
        frames.append(dataset.loc[n:n+1].to_numpy())
        n += 1
    else:
        return frames


In [241]:
gest_frames = frame()

In [247]:
len(gest_frames)

5460

### Peprocessing 3 > FFT, feature extraction and feature vector 

In [245]:
def fft_transform(frame_data):
    
    fft_transform.feature_set = np.array([])
    
    for frame in frame_data:
        
        frame_feature_set = np.array([])
        
        fft_frame = np.fft.fft(frame, axis=0)
#         fft_transform.fft_data.append(fft_frame)
        
#         n = frame.size
#         freq = np.fft.fftfreq(n, d=0.01)
#         fft_transform.fft_freq.append(freq)
        
#         fft_ps = np.abs(fft_frame)**2
#         fft_transform.fft_power_spec.append(fft_ps)
        
        mean = np.mean(fft_frame, axis=0)
        frame_feature_set = np.concatenate((frame_feature_set, mean), axis=0)
        
        
        energy = (np.sum((fft_frame**2),axis=0))/len(fft_frame)
        frame_feature_set = np.concatenate((frame_feature_set, energy), axis=0)
        
        
        stdev = np.std(fft_frame, axis=0)
        frame_feature_set = np.concatenate((frame_feature_set, stdev), axis=0)
        
        
        correlation = np.corrcoef(fft_frame, rowvar=False).reshape(3,3)
        frame_feature_set = np.concatenate((frame_feature_set.reshape(3,3), correlation), axis=0)
        
        fft_transform.feature_set = np.concatenate((frame_feature_set.flatten(), fft_transform.feature_set), axis=0)
    
#  make a simple array         

    return fft_transform.feature_set

In [246]:
fft_transform(gest_frames)

array([ 1.70068359+0.j,  0.11621094+0.j, -1.56640625+0.j, ...,
       -1.        +0.j, -1.        +0.j,  1.        +0.j])

### Training data set 

In [250]:
n = int((fft_transform.feature_set.size)/180)
axis = 3
feature_10axis = 60
fft_transform.feature_set = fft_transform.feature_set.reshape(n,(feature_10axis*axis))
X = np.abs(fft_transform.feature_set)
y = labels

In [334]:
y.shape

(546,)

In [335]:
X.shape

(546, 180)

In [315]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [316]:
C = 0.1
clf = svm.SVC(kernel='poly',C=C)
clf.fit(X_train, y_train)

/Users/martinantos/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [317]:
y_pred = clf.predict(X_test)

In [318]:
print("accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

accuracy: 0.9272727272727272
Precision: 0.9803921568627451
Recall: 0.8771929824561403
